# セットアップ

[リンクテキスト](https://qiita.com/kinakomochi_/items/95990d139f5e5e57fd67)

In [ ]:
!pip install -U ginza ja_ginza_electra

In [ ]:
!pip install -U ginza ja-ginza

In [ ]:
import pkg_resources, imp
imp.reload(pkg_resources)

In [ ]:
import spacy

nlp = spacy.load('ja_ginza_electra')  # または 'ja_ginza'（インストール済みモデル名に応じて）
doc = nlp('文書の形態素解析をしてみたよ')

for sent in doc.sents:
    for token in sent:
        print(
            token.i,                        # トークンのインデックス
            token.orth_,                    # 表層形
            token.lemma_,                   # 基本形
            token.norm_,                    # 正規化形
            token.morph.get("Reading"),     # 読み
            token.pos_,                     # 品詞
            token.morph.get("Inflection"),  # 活用形
            token.tag_,                     # 詳細品詞タグ
            token.dep_,                     # 係り受け関係
            token.head.i                    # 係り先のトークンインデックス
        )

In [ ]:
import spacy

In [ ]:
!pip install nbstripout

In [ ]:
!nbstripout 100本ノック第4章.ipynb

# 33
文章textに係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

In [ ]:
import spacy

nlp=spacy.load("ja_ginza_electra")
text="""
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

doc=nlp(text)

for token in doc:
    print(f"{token.text}\t{token.head.text}")

In [ ]:
import spacy
import pandas as pd
from IPython.display import display

# GiNZA ELECTRAモデルの読み込み
nlp = spacy.load("ja_ginza_electra")

# 対象テキスト
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

# GiNZAで係り受け解析を実行
doc = nlp(text)

# 結果をリストで保持
rows = []
for token in doc:
    rows.append({
        "表層形": token.text,
        "原形": token.lemma_,
        "品詞": token.pos_,
        "係り受け関係": token.dep_,
        "係り先の単語": token.head.text,
        "係り元の位置": token.i,
        "係り先の位置": token.head.i
    })

# pandasのDataFrameに変換
df = pd.DataFrame(rows)

# Colabで見やすく表示（テーブル形式）
display(df.style.set_table_styles([
    {'selector': 'thead th', 'props': [('background-color', '#f0f0f0'), ('color', 'black')]},
    {'selector': 'tbody td', 'props': [('text-align', 'center')]}
]).set_properties(**{'border': '1px solid gray', 'padding': '5px'}))

# 34:主述の関係
文章textにおいて、「メロス」が主語であるときの述語を抽出せよ。
[出力の読み方(主語がどれか)](https://note.com/npaka/n/n5c3e4ca67956#PrfvI)

In [ ]:
# import spacy

nlp=spacy.load("ja_ginza")

text """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

doc=nlp(text)

predicates=[]

for sent in doc.sents:
    for token in sent:
        if token.text=="メロス" and token.dep_=="nsubj":
            head=token.head
            if head.pos_ in ("VERB","AUX"):
                predicates.append(head.text)
            elif head.pos_=="NOUN":
                cop_found=None
                fixed_found=None
                for child in head.children:
                    if child.dep_ == "cop":
                        cop_found = child
                        for grandchild in child.children:
                            if grandchild.dep_ == "fixed":
                                fixed_found = grandchild
                if cop_found and fixed_found:
                    predicates.append(f"{head.text}{cop_found.text}{fixed_found.text}")  # e.g., 牧人である
                elif cop_found:
                    predicates.append(f"{head.text}{cop_found.text}")  # 牧人で
                else:
                    predicates.append(head.text)  # 名詞単独

print(sorted(set(predicates)))

# 35:係り受け木
「メロスは激怒した。」の係り受け木を可視化せよ。

[【初心者向け】自然言語処理ツール「GiNZA」を用いた言語解析（形態素解析からベクトル化まで）](https://qiita.com/cove_ht/items/63ffdd8ff237d4845566#%E3%81%AF%E3%81%98%E3%82%81%E3%81%AB)

In [ ]:
from spacy import displacy
from IPython.display import display, HTML
import spacy

nlp=spacy.load("ja_ginza")
text="メロスは激怒した。"
doc=nlp(text)

html=displacy.render(doc,style="dep",options={"compact":True})
display(HTML(html))

# 36~39
問題36から39までは、Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzをコーパスと見なし、統計的な分析を行う。

1行に1記事の情報がJSON形式で格納される
各行には記事名が”title”キーに、記事本文が”text”キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される
ファイル全体はgzipで圧縮される


[Colab環境でMeCabを使う3行](https://qiita.com/Ninagawa123/items/c90cccb453e2a6fc4466)

In [ ]:
! pip install mecab-python3 unidic-lite
import MeCab
print(MeCab.Tagger().parse("これはテストです"))

# 36:単語の出現頻度
まず、第3章の処理内容を参考に、Wikipedia記事からマークアップを除去し、各記事のテキストを抽出せよ。そして、コーパスにおける単語（形態素）の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

[MeCabの出力形式を整理して、Pandasで扱えるようにした](https://qiita.com/hasoya/items/0561bb1481a648aa8e6e)

In [ ]:
import gzip
import json
import re
from collections import Counter
import MeCab

tagger = MeCab.Tagger()
file_path = "/content/drive/MyDrive/jawiki-country.json.gz"
word_counter = Counter()

with gzip.open(file_path, mode='rt', encoding='utf-8') as f:
    for line in f:
        article = json.loads(line)
        text = article.get("text", "")
        clean_text = re.sub(r'\[\[.*?\]\]', '', text)
        clean_text = re.sub(r"''+", '', clean_text)
        node = tagger.parseToNode(clean_text)
        while node:
            surface = node.surface
            features = node.feature.split(',')
            pos = features[0]
            if not pos.startswith("補助記号") and not pos.startswith("助詞") and not pos.startswith("助動詞"):
                word_counter[surface] += 1
            node = node.next

for word, freq in word_counter.most_common(20):
    print(f"{word}\t{freq}")

In [ ]:
# 1. Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 2. クリーンアップツールをインストール
!pip install nbstripout -q

# 3. ★★★ 自分のノートブックのパスに書き換える ★★★
NOTEBOOK_PATH = "/content/drive/My Drive/Colab Notebooks/Your_Notebook_Name.ipynb"

# 4. nbstripoutを実行して、ノートブックから不要なメタデータを削除
!nbstripout "{NOTEBOOK_PATH}"

print(f"\n✅ クリーンアップ完了: {NOTEBOOK_PATH}")
print("このファイルをGoogle DriveからダウンロードしてGitHubにアップロードしてください。")